In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import re


# Importing the data from the data set

In [47]:
df = pd.read_csv(r'C:\Users\gdmarques\OneDrive - CRITICAL SOFTWARE, S.A\Documents\Customer Segmentation\customer-segmentation\customer_segmentation\data\bank_transactions.csv')
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


# Data Cleanup and Preprocess

Dropping missing data

In [48]:
df.dropna(inplace = True)

Check for duplicated values

In [49]:
df.duplicated().sum()

np.int64(0)

Both the date of birth and transaction date are in weird types, so I will convert them to datetime format so it is easier to interpret

In [50]:
def convert_date(date_str):
    date = pd.to_datetime(date_str, format='%d/%m/%y', errors='coerce')
    # Adjust century for dates that are incorrectly in the 2000s
    if date.year > pd.Timestamp.now().year:
        date = date - pd.DateOffset(years=100)
    return date

df['TransactionDate'] = df['TransactionDate'].apply(convert_date)
df['CustomerDOB'] = df['CustomerDOB'].apply(convert_date)


I will create a new column called "CustomerAge" because the age of the customer when each transaction was made. 
For a bank it is relevant to have this information, because it provides insights into spending behavior, product needs, risk assessment, and customer engagement strategies. 
By analyzing this information, banks can enhance their offerings, improve customer experience, and foster long-term relationships with customers.

In [51]:
df['CustomerAge'] =df['TransactionDate'].dt.year - df['CustomerDOB'].dt.year

In [52]:
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0,22.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0,59.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,20.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,43.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,181156,1762.5,28.0


The dataset has to much data and a few , and has some inconsistencies in the CusLocation column.

To overcome theses complications, I decided to keep only the lines where the location is repeated at least 7500 times, to make the dataset more consistent and relevant at the same time.

In [55]:
location_counts = df['CustLocation'].value_counts()

print(location_counts)

CustLocation
MUMBAI                  101997
NEW DELHI                84143
BANGALORE                81330
GURGAON                  73594
DELHI                    70549
                         ...  
SAHIBGANJ                    1
WEST WAGLE I E THANE         1
VIRAR MUMBAI                 1
CHUNCHURA HOOGHLY            1
MAHINDER GARH                1
Name: count, Length: 9275, dtype: int64


In [56]:
df_test = df.sort_values(by = 'CustLocation')

df_test.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
999576,T999577,C3428855,1993-02-02,F,(154) BHASKOLA FARIDABAD,49388.09,2016-09-14,131252,1218.72,23.0
254706,T254707,C5028866,1993-02-02,F,(154) BHASKOLA FARIDABAD,49388.09,2016-08-07,214729,30.00,23.0
246731,T246732,C4328852,1993-02-02,F,(154) BHASKOLA FARIDABAD,49388.09,2016-08-07,190025,99.00,23.0
248110,T248111,C3028850,1993-02-02,F,(154) BHASKOLA FARIDABAD,49388.09,2016-08-07,161607,50.00,23.0
686004,T686005,C2728824,1993-02-02,F,(154) BHASKOLA FARIDABAD,49388.09,2016-08-30,223715,3383.13,23.0


In [64]:
location_counts = df['CustLocation'].value_counts()


df = df[df['CustLocation'].isin(location_counts[location_counts >= 7000].index)]



In [67]:
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,20.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,43.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,181156,1762.5,28.0
6,T7,C7126560,1992-01-26,F,MUMBAI,973.46,2016-08-02,173806,566.0,24.0
7,T8,C1220223,1982-01-27,M,MUMBAI,95075.54,2016-08-02,170537,148.0,34.0


# Adding relevant information to the dataset

A relevant information I should gather is the frequency that people make transactions, the freshness of customer activity and the willingness to spend.

This is relevant information for banks, because this provides answer to some questions like:
 - Who are the best clients?
 - Who are the most loyal customers?
 - Which client segment will react favorably to your current/next campaign?

## Recency

To calculate Recency will start by calculating the first and last transaction

In [69]:
df['TransactionDate1']=df['TransactionDate'] # ==> to calculate the minimum (first transaction)
df['TransactionDate2']=df['TransactionDate'] # ==> to calculate the maximum (last transaction)

Will create a new dataframe for this purpose

In [70]:
MRF_df = df.groupby("CustomerID").agg({
                                        "TransactionID" : "count",
                                        "CustGender" : "first",
                                        "CustLocation":"first",
                                        "CustAccountBalance"  : "mean",
                                        "TransactionAmount (INR)" : "mean",
                                        "CustomerAge" : "median",
                                        "TransactionDate2":"max",
                                        "TransactionDate1":"min",
                                        "TransactionDate":"median"
                        })

MRF_df = MRF_df.reset_index()
MRF_df.head()

,CustomerID,TransactionID,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),CustomerAge,TransactionDate2,TransactionDate1,TransactionDate
0,C1010011,2,F,NOIDA,76340.635,2553.0,28.5,2016-09-26,2016-08-09,2016-09-02
1,C1010012,1,M,MUMBAI,24204.490,1499.0,22.0,2016-08-14,2016-08-14,2016-08-14
2,C1010014,2,F,MUMBAI,100112.950,727.5,28.0,2016-08-07,2016-08-01,2016-08-04
3,C1010024,1,M,KOLKATA,87058.650,5000.0,51.0,2016-08-18,2016-08-18,2016-08-18
4,C1010028,1,F,DELHI,296828.370,557.0,28.0,2016-08-29,2016-08-29,2016-08-29


In [72]:
MRF_df.drop(columns=["CustomerID"],inplace=True)

The dataset already has a transaction column but is identified as TransactionID and it accumulates the number of transactions made by a certain Customer, so I will just change the name of the column

In [73]:
MRF_df.rename(columns={"TransactionID":"Frequency"},inplace=True)

In [74]:
MRF_df['Recency']=MRF_df['TransactionDate2']-MRF_df['TransactionDate1']

In [75]:
print(MRF_df['Recency'])

0        48 days
1         0 days
2         6 days
3         0 days
4         0 days
           ...  
577749    0 days
577750    0 days
577751    0 days
577752    0 days
577753    0 days
Name: Recency, Length: 577754, dtype: timedelta64[ns]


To calculate the recency I had to subtract 2 dates and the result will always be in days, since it is not usefull in this datatype for what I am doing, I will convert it to string and remove the "days" part of the values

In [77]:
MRF_df['Recency']=MRF_df['Recency'].astype(str)
x='18 day'
re.search('\d+',x).group()

'18'

In [78]:
MRF_df['Recency']=MRF_df['Recency'].apply(lambda x :re.search('\d+',x).group())
MRF_df['Recency']=MRF_df['Recency'].astype(int)

As seen before some entries are with 0 days, that means a customer made a recent one time transaction, so I will convert 0 to 1

In [79]:
def rep_0(i):
    if i==0:
        return 1
    else:
        return i
MRF_df['Recency']=MRF_df['Recency'].apply(rep_0)

TransactionDate1 and TransactionDate2 are no longer useful so I will drop them from the dataframe now

In [80]:
MRF_df.drop(columns=["TransactionDate1","TransactionDate2"],inplace=True)

In [81]:
MRF_df.head()

,Frequency,CustGender,CustLocation,CustAccountBalance,TransactionAmount (INR),CustomerAge,TransactionDate,Recency
0,2,F,NOIDA,76340.635,2553.0,28.5,2016-09-02,48
1,1,M,MUMBAI,24204.490,1499.0,22.0,2016-08-14,1
2,2,F,MUMBAI,100112.950,727.5,28.0,2016-08-04,6
3,1,M,KOLKATA,87058.650,5000.0,51.0,2016-08-18,1
4,1,F,DELHI,296828.370,557.0,28.0,2016-08-29,1


Now I can move on to Exploratory Data Analysis on the dataset